Tensor→Numpy配列(Conda Cellで)

In [ ]:
import torch
import numpy as np
import os
feature_path = "experience/2023_4_18_32pix/trainfeat.pth"
label_path = "experience/2023_4_18_32pix/trainlabels.pth"

feature_data = torch.load(feature_path)
label_data = torch.load(label_path)
feature_data = feature_data.to('cpu').detach().numpy().copy()
label_data  = label_data .to('cpu').detach().numpy().copy()
folder_path = os.path.dirname(feature_path)

print(feature_data.shape)
print(label_data .shape)
np.save(os.path.join(folder_path, "feature.npy"), feature_data)
np.save(os.path.join(folder_path, "label.npy"), label_data)

numpy配列ロード→学習(Train Data)

In [ ]:
# from cuml import svm #GPU version
from sklearn import svm #cpu Version
import numpy as np

#
CUT_PARAM = False

feature_data = np.load(os.path.join(folder_path, "feature.npy"))
label_data = np.load(os.path.join(folder_path, "label.npy"),)
print(np.unique(label_data))
print(feature_data.shape)
print(label_data.shape)

#split day array by label.
for i, label in enumerate(label_data):
    if(label==0):
        length0 = i
    if(label==1):
        length3 = i - length0   
    if(label==2):
        length5 = i - length0 - length3
        
print("length", length0, length3, length5)

day0, feature_data = np.split(feature_data, [length0])
day3, feature_data = np.split(feature_data, [length3])
day5, feature_data = np.split(feature_data, [length5])
day7 = feature_data

###split
import random
random.shuffle(day0)
random.shuffle(day3)
random.shuffle(day5)
random.shuffle(day7)

# 7dayのdataを削る
if(CUT_PARAM):
    print("Before_NUM:", len(day0)+len(day3)+len(day5)+len(day7))
    print("Before_ALL:", day0.shape, day3.shape, day5.shape, day7.shape)
    _, day7 = np.split(day7, [len(day7)//2])

day0_valid, day0_train = np.split(day0, [len(day0)//5])
day3_valid, day3_train = np.split(day3, [len(day3)//5])
day5_valid, day5_train = np.split(day5, [len(day5)//5])
day7_valid, day7_train = np.split(day7, [len(day7)//5])

print("NUM:", len(day0)+len(day3)+len(day5)+len(day7))
print("ALL:", day0.shape, day3.shape, day5.shape, day7.shape)
print("Valid:", day0_valid.shape, day3_valid.shape, day5_valid.shape, day7_valid.shape)
print("Train:", day0_train.shape, day3_train.shape, day5_train.shape, day7_train.shape)

Learning

In [ ]:
nu_rate = 0.1
kernel = "rbf"

# 1-class SVMを訓練する
svm_model0 = svm.OneClassSVM(kernel=kernel, nu=nu_rate)
svm_model0.fit(day0_train)

svm_model3 = svm.OneClassSVM(kernel=kernel,nu=nu_rate)
svm_model3.fit(day3_train)

svm_model5 = svm.OneClassSVM(kernel=kernel,nu=nu_rate)
svm_model5.fit(day5_train)

svm_model7 = svm.OneClassSVM(kernel=kernel,nu=nu_rate)
svm_model7.fit(day7_train)

print(svm_model0.intercept_, svm_model3.intercept_, svm_model5.intercept_, svm_model7.intercept_, )

# モデルの保存
import pickle
with open(os.path.join(folder_path,'svm_model0.pkl'), 'wb') as f:
    pickle.dump(svm_model0, f)
with open(os.path.join(folder_path,'svm_model3.pkl'), 'wb') as f:
    pickle.dump(svm_model3, f)
with open(os.path.join(folder_path,'svm_model5.pkl'), 'wb') as f:
    pickle.dump(svm_model5, f)
with open(os.path.join(folder_path,'svm_model7.pkl'), 'wb') as f:
    pickle.dump(svm_model7, f)

検証実験(valid Data)　決定関数の値と閾値の距離を算出する

In [ ]:
#day0の画像特徴量を各モデルで推論してみる.
import numpy as np
import matplotlib.pyplot as plt

arr = ["0", "3", "5", "7"]
color = ["r", "b", "#ffff33", "#ff7f00"]

for d in arr:#データごとに
    i = 0
    gap = []
    val = []
    result = []
    for model_day in arr:#モデルごとに
        intercept = globals()["svm_model" + model_day].intercept_
        for data in globals()["day" + d + "_valid"]:
            data = data.reshape(1, -1)
            value = globals()["svm_model" + model_day].decision_function(data)
            val.append(value)
            gap.append(value-intercept)
            result.append(globals()["svm_model" + model_day].predict(data))

    #色分けしてグラフ化
    data_num = len(globals()["day" + d + "_valid"])
    y0 = list(range(1, data_num+1))
    val0, val = np.split(val, [data_num])
    
    y3 = list(range(data_num+2, 2*data_num+2))
    val3, val = np.split(val, [data_num])
    
    y5 = list(range(2*data_num+3, 3*data_num+3))
    val5, val = np.split(val, [data_num])

    y7 = list(range(3*data_num+4, 4*data_num+4))
    val7, val = np.split(val, [data_num])
    
    #plt.plot(result)
    print("***************************************************************")
    print("Inputday"+ str(d) +"-0day-3day-5day-7dayModel")
    plt.scatter(y0, val0, color=color[0])
    plt.scatter(y3, val3, color=color[1])
    plt.scatter(y5, val5, color=color[2])
    plt.scatter(y7, val7, color=color[3])
    plt.savefig(os.path.join(folder_path, "Inputday"+ str(d) +"-0day-3day-5day-7dayModel.png"))
    #plt.show()
    

Testデータで確認

In [ ]:
import numpy as np
#model path
#model_folder = "./model/64crop_0412/"
#data_folder = "./Raw_feature_data/64_Test_0411"
feature_data = np.load(os.path.join(folder_path ,"feature.npy"))
label_data = np.load(os.path.join(folder_path , "label.npy"))
print(feature_data.shape)
print(label_data.shape)
print(np.unique(label_data))


#model load
import pickle
with open(folder_path +'svm_model0.pkl', 'rb') as f:
    svm_model0 = pickle.load(f)
with open(folder_path +'svm_model3.pkl', 'rb') as f:
    svm_model3 = pickle.load(f)
with open(folder_path +'svm_model5.pkl', 'rb') as f:
    svm_model5 = pickle.load(f)
with open(folder_path +'svm_model7.pkl', 'rb') as f:
    svm_model7 = pickle.load(f)

#data labelごとに分割
for i, label in enumerate(label_data):
    if(label==0):
        length0 = i
    if(label==1):
        length3 = i - length0   
    if(label==2):
        length5 = i - length0 - length3 

day0, feature_data = np.split(feature_data, [length0])
day3, feature_data = np.split(feature_data, [length3])
day5, feature_data = np.split(feature_data, [length5])
day7 = feature_data

print(length0,length3,length5)
print(day0.shape, day3.shape, day5.shape, day7.shape)


推論＆プロット（TEST）

In [10]:
#プロット
import numpy as np
import matplotlib.pyplot as plt

arr = ["0", "3", "5", "7"]
color = ["r", "b", "#ffff33", "#ff7f00"]


for d in arr:#データごとに
    i = 0
    gap = []
    val = []
    result = []
    for model_day in arr:#モデルごとに
        intercept = globals()["svm_model" + model_day].intercept_
        for data in globals()["day" + d]:
            data = data.reshape(1, -1)
            value = globals()["svm_model" + model_day].decision_function(data)
            val.append(value)
            gap.append(value-intercept)
            result.append(globals()["svm_model" + model_day].predict(data))

    #色分けしてグラフ化
    data_num = len(globals()["day" + d])
    y0 = list(range(1, data_num+1))
    val0, val = np.split(val, [data_num])
    
    y3 = list(range(data_num+2, 2*data_num+2))
    val3, val = np.split(val, [data_num])
    
    y5 = list(range(2*data_num+3, 3*data_num+3))
    val5, val = np.split(val, [data_num])

    y7 = list(range(3*data_num+4, 4*data_num+4))
    val7, val = np.split(val, [data_num])
    
    #plt.plot(result)
    print("***************************************************************")
    print("Inputday"+ str(d) +"-0day-3day-5day-7dayModel")
    plt.scatter(y0, val0, color=color[0])
    plt.scatter(y3, val3, color=color[1])
    plt.scatter(y5, val5, color=color[2])
    plt.scatter(y7, val7, color=color[3])
    plt.savefig(os.path.join(folder_path,"TEST_Inputday"+ str(d) +"-0day-3day-5day-7dayModel.png"))
    #plt.show()

超平面ぷろっと

In [ ]:
#https://axa.biopapyrus.jp/machine-learning/outlier-detection/one-class-svm.html

import numpy as np
X_outliers = np.random.uniform(low=-0.1, high=0.15, size=(20, 2))
#xx, yy = np.meshgrid(np.linspace(-5, 5, 768), np.linspace(-5, 5, 768))
#Z = svm_model0.decision_function(np.c_[xx.ravel(), yy.ravel()])
#Z = Z.reshape(xx.shape)

# サポートベクターを取得
#sv1 = svm_model0.support_vectors_

##サポートベクターをプロット
plt.scatter(day0_train[:, 0], day0_train[:, 1], color='black', edgecolors='k', label='Train-data')
plt.scatter(day0_valid[:, 0], day0_valid[:, 1], color='red', edgecolors='k', label='Valid-data')
#plt.scatter(sv1[:, 0], sv1[:, 1], color='r', label='supprt_vector')
c = plt.scatter(day3_valid[:, 0], day3_valid[:, 1], c='blue', edgecolors='k', label='3day-data')
d = plt.scatter(day5_valid[:, 0], day5_valid[:, 1], c='yellow', edgecolors='k', label='5day-data')
e = plt.scatter(day7_valid[:, 0], day7_valid[:, 1], c='orange', edgecolors='k', label='7day-data')

plt.legend()
plt.savefig("day0_learning_SVM.png")
plt.show()